# Day 10: Syntax Scoring
https://adventofcode.com/2021/day/10

### Part 1, Detect syntax errors

Rules,
- If a chunk opens with (, it must close with ).
- If a chunk opens with [, it must close with ].
- If a chunk opens with {, it must close with }.
- If a chunk opens with <, it must close with >.

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

In [1]:
import os
from typing import Optional, Tuple

In [2]:
# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_10'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_10'
) -> str:
    return os.path.join(data_directory, file_loc)

def read_input(input_file:str) -> list[str]:
    lines: list[str] = list()
    if os.path.exists(input_file):
        with open(input_file) as f:    
            for line in f:
                if line.rstrip():
                    lines.append(line.strip())
    return lines


In [3]:
def has_syntax_errors(line: str, print_errors=False) -> Tuple[bool, Optional[str]]:
    """
    Returns True if there is a syntax error and the first offending character,
    otherwise False and None
    Tuple(bool, str)
    """

    # dictionary for our valid open/close pairs
    valid_pairs: dict = {
        '(': ')',
        '[': ']',
        '{': '}',
        '<': '>'
    }
    syntax_stack: list = list()

    for ch in line:
        if ch in valid_pairs.keys():
            syntax_stack.append(ch)
        
        elif ch in valid_pairs.values():   
            # closer without an prior opener
            if len(syntax_stack) == 0:
                return (True, ch)

            # our closer should be the pair of the last opener.        
            stack_ch = syntax_stack.pop()
            if ch != valid_pairs.get(stack_ch, ''):
                if print_errors:
                    print(f"{line} - Expected {valid_pairs.get(stack_ch, '')}, but found {ch} instead.")
                return (True, ch)

    # len(syntax_stack) > 0 then it is incomplete.

    return False, None

def score_input(lines: list[str]) -> int:
    """
    Loop through input "lines" and sum error score according the scheme.
    """
    error_score: dict = {
        ')': 3,
        ']': 57,
        '}': 1197,
        '>': 25137
    }
    sum = 0
    for line in lines:
        has_error, ch = has_syntax_errors(line)
        if has_error:
            sum = sum + error_score.get(ch, 0)
    return sum

In [4]:
# Test various failure examples.
assert has_syntax_errors("{([(<{}[<>[]}>{[]{[(<()>", print_errors=True) == (True, '}')
assert has_syntax_errors("[[<[([]))<([[{}[[()]]]", print_errors=True) == (True, ')')
assert has_syntax_errors("[{[{({}]{}}([{[{{{}}([]", print_errors=True) == (True, ']')
assert has_syntax_errors("[<(<(<(<{}))><([]([]()", print_errors=True) == (True, ')')
assert has_syntax_errors("<{([([[(<>()){}]>(<<{{", print_errors=True) == (True, '>')

{([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
[[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
[{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
[<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
<{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.


In [5]:
# Test the input data provided.
assert score_input(read_input(test_input_location())) == 26397

In [6]:
score_input(read_input(input_location()))

215229

### Part 2 - Incomplete lines.
We detect and complete incomplete lines.  We also perform the specified scoring algorithm on the lines to answer the question.

In [50]:
def is_incomplete(line: str, print_errors=False) ->Tuple[bool, Optional[str]]:
    """
    Returns True if the line is incomplete and the missing string elements, otherwise 
    returns False and None even when the line has syntax errors.
    Tuple(bool, str)
    """

    if has_syntax_errors(line)[0]:
        return False, None  # it is not incomplete, it is broken.

    # dictionary for our valid open/close pairs
    valid_pairs: dict = {
        '(': ')',
        '[': ']',
        '{': '}',
        '<': '>'
    }
    syntax_stack: list = list()

    for ch in line:
        if ch in valid_pairs.keys():
            syntax_stack.append(ch)
        
        elif ch in valid_pairs.values():   
            # our closer should be the pair of the last opener.        
            stack_ch = syntax_stack.pop()
            if ch != valid_pairs.get(stack_ch, ''):
                return False, None

    # check for in_completeness
    if len(syntax_stack) > 0:
        required_to_complete = ""
        for ch in reversed(syntax_stack):
            required_to_complete = required_to_complete + valid_pairs.get(ch, '')

        if print_errors:
            print(f"{line} - Complete by adding {required_to_complete}")
        
        return True, required_to_complete

    return False, None

def score_input_completeness(line: str) -> int:
    """
    Scores a single line. Non-incomplete lines are skipped.
    The specified scoring algorithm is implemented here.
    """
    error_score: dict = {
        ')': 1,
        ']': 2,
        '}': 3,
        '>': 4
    }
    
    total_score = 0
    incomplete, to_complete = is_incomplete(line)
    if incomplete:
        for ch in to_complete:
            total_score = total_score * 5 + error_score.get(ch, 0)

    return total_score

def score_document(lines: list[str]) -> int:
    """
    Scores the each line in the document. Non-incomplete lines are skipped.  We return the middle score. 
    """
    scores: list[int] = []
    for line in lines:
        score = score_input_completeness(line)
        if score > 0:
            scores.append(score)

    if len(scores):
        scores = sorted(scores)
        mid_point_idx = round( (len(scores)-1) / 2 )
        return scores[mid_point_idx]
    
    return 0


In [51]:
# Test incomplete check and completion logic
assert is_incomplete("[({(<(())[]>[[{[]{<()<>>", print_errors=True) == (True, "}}]])})]")
assert is_incomplete("[(()[<>])]({[<{<<[]>>(", print_errors=True) == (True, ")}>]})")
assert is_incomplete("(((({<>}<{<{<>}{[]{[]{}", print_errors=True) == (True, "}}>}>))))")
assert is_incomplete("{<[[]]>}<{[{[{[]{()[[[]", print_errors=True) == (True, "]]}}]}]}>")
assert is_incomplete("<{([{{}}[<[[[<>{}]]]>[]]", print_errors=True) == (True, "])}>")

[({(<(())[]>[[{[]{<()<>> - Complete by adding }}]])})]
[(()[<>])]({[<{<<[]>>( - Complete by adding )}>]})
(((({<>}<{<{<>}{[]{[]{} - Complete by adding }}>}>))))
{<[[]]>}<{[{[{[]{()[[[] - Complete by adding ]]}}]}]}>
<{([{{}}[<[[[<>{}]]]>[]] - Complete by adding ])}>


In [52]:
# Test line scoring
assert score_input_completeness("[({(<(())[]>[[{[]{<()<>>") == 288957
assert score_input_completeness("[(()[<>])]({[<{<<[]>>(") == 5566  
assert score_input_completeness("<{([{{}}[<[[[<>{}]]]>[]]") == 294

In [54]:
# Test document scoring.
assert score_document(read_input(test_input_location())) == 288957

In [55]:
# RUN ON PROD DATA TO FIND THE ANSWER
score_document(read_input(input_location()))

1105996483